In [ ]:
###Epic PGSを実装します．
#### 不確実性集合，コスト関数集合の作り方が非常に良くないので改良します．

import random
import re
import numpy as np
from sympy import Q, comp
S = 3  # 状態数
A = 2  # 行動数
N = 3  # 制約数+1
T = 100 # タイムステップ数
U_num = 3  # 不確実性集合の数

policy = np.array([S, A])  

J_n_hat_b = np.zeros(N)  # 目的関数の推定値

n_T = np.array([T])

gamma = 0.99  # 割引率

pi = np.zeros((S, A))  # 初期ポリシーのパラメータ
### ポリシーのパラメータを初期化
pi = np.random.rand(S, A)  # ランダムに初期化
pi = pi / np.sum(pi, axis=1, keepdims=True)  # 各状態の行動確率を正規化
pi = pi.reshape(S * A)  # ポリシーを1次元配列に変形

mu = np.random.rand(S)  # 初期状態の分布
mu = mu / np.sum(mu)  # 状態分布を正規化

H = 1 /(1 - gamma)  # ホライズンの計算
H = int(np.round(H))  # ホライズンを整数に変換



def make_U():
    """
    不確実性集合を作成します．
    各状態遷移確率行列をランダムに生成し、正規化します．
    """
    U = []
    for _ in range(U_num):
        P = np.random.rand(S, A, S)  # ランダムに初期化
        P = P / np.sum(P, axis=2, keepdims=True)  # 各状態の行動確率を正規化
        U.append(P)
    return U

U = make_U()  # 不確実性集合を作成

def make_C_and_Q_pi_b():
    """
    コスト関数の集合を作成します．
    各コスト関数をランダムに生成します．
    """
    C = []
    Q = []
    B = []
    for _ in range(N):
        c = np.random.rand(S, A)  # ランダムに初期化
        C.append(c)
        Q_pi = np.zeros((S, A))
        Q.append(Q_pi) 
        #0からHの間の値でbを定義
        b = random.randint(0, H)
        B.append(b)
    return C,Q,B

C,Q,B = make_C_and_Q_pi_b()  # コスト関数の集合を作成


def Projected_Policy_Gradient_Subroutine(bo,alpha,T,J_n_hat,subgradient_J_n_hat):
    """
    Epic PGSのサブルーチンを実装します．
    
    Parameters:
    bo : np.ndarray
        初期状態の分布
    alpha : float
        学習率
    T : int
        エピソードの長さ
    J_n_hat : np.ndarray
        目的関数の推定値
    subgradient_J_n_hat : np.ndarray
        目的関数のサブグラディエント

    Returns:
    np.ndarray
        更新されたポリシーのパラメータ
    
    処理は，主に，Jn_hat(pi_t) - bnを最大化する制約を求める関数+nに対して勾配を求め，射影勾配を出力して方策更新する関数を用います．
    
    """
    for t in range(T):
        n_T[t] = compute_argmax_n(J_n_hat, bo)
        policy = compute_projection(policy, subgradient_J_n_hat[n_T[t]], alpha)
    return policy  # 更新されたポリシーのパラメータを返す

def compute_J_n_hat(pi, n):
    """
    目的関数を計算します．
    J_c_P(pi)
    """
    pass


def compute_argmax_n(J_n_hat,b):
    for n in range(N):
        J_n_hat_b[n] = J_n_hat[n] - b[n]
    return np.argmax(J_n_hat_b)  # 最大値を持つインデックスを返す

def compute_projection(pi,g,alpha):
    """
    射影勾配を計算します．
    
    Parameters:
    pi : np.ndarray
        現在のポリシーのパラメータ
    g : np.ndarray
        勾配
    alpha : float
        学習率

    Returns:
    np.ndarray
        射影されたポリシーのパラメータ
    """
    pass


def compute_dot_pi_Q(s,Q_pi):
    """
    ポリシーとQ関数のドット積を計算します．
    Parameters:
    s : int
        状態のインデックス
    Returns:
    float
        ポリシーとQ関数のドット積
    """
    return np.dot(pi[s:s+A], Q_pi[s, :])  # ポリシーとQ関数のドット積を計算

def compute_sum_dot_pi_Q(Q_pi):
    """
    ポリシーとQ関数のドット積の和を計算します．
    
    Parameters:
    s : int
        状態のインデックス
    
    Returns:
    float
        ポリシーとQ関数のドット積の和
    """
    results = np.zeros(S)  # 各状態のドット積の結果を格納する配列
    for s in range(S):
        results[s] = compute_dot_pi_Q(s,Q_pi)  # 各状態のドット積を計算
    return results  # 各状態のドット積の結果を返す

def compute_sum_P_dot_dot_pi_Q(s,a,P,Q_pi):
    return np.dot(P[s, a, :], compute_sum_dot_pi_Q(Q_pi))  # 状態遷移確率とポリシーのドット積を計算

def compute_Q_pi_c_P(s, a, P,Q_pi,c):
    """
    Q関数を計算します．
    
    Parameters:
    s : int
        状態のインデックス
    a : int
        行動のインデックス
    
    Returns:
    float
        Q関数の値
    """
 
    return c[s,a] + gamma * compute_sum_P_dot_dot_pi_Q(s, a, P,Q_pi)  # 状態遷移確率とポリシーのドット積を計算してQ関数を求める

def compute_Q_pi_c_P_matrix(P,Q_pi,c):
    """
    Q関数の行列を計算します．
    
    Returns:
    np.ndarray
        Q関数の行列
    """
    for s in range(S):
        for a in range(A):
            Q_pi[s, a] = compute_Q_pi_c_P(s, a, P,Q_pi,c)  # 各状態と行動のQ関数を計算
    return Q_pi  # Q関数の行列を返す


def compute_V_pi_c_P(s,P,Q_pi,c):
    """
    V関数を計算します．
    
    Parameters:
    s : int
        状態のインデックス
    
    Returns:
    float
        V関数の値
    """
    Q_pi = compute_Q_pi_c_P_matrix(P,Q_pi,c)  # Q関数の行列を計算
    return np.dot(pi[s:s+A], Q_pi[s, :])  # ポリシーとQ関数のドット積を計算してV関数を求める

def compute_V_pi_c_P_matrix(P,Q_pi,c):
    """
    V関数の行列を計算します．
    
    Returns:
    np.ndarray
        V関数の行列
    """
    results = np.zeros(S)  # 各状態のV関数の結果を格納する配列
    for s in range(S):
        results[s] = compute_V_pi_c_P(s,P,Q_pi,c)  # 各状態のV関数を計算
    return results  # 各状態のV関数の結果を返す

def compute_J_c_P(P,Q_pi,c):
    """
    目的関数を計算します．
    
    Returns:
    float
        目的関数の値
    """
    return np.dot(mu, compute_V_pi_c_P_matrix(P,Q_pi,c))  # 初期状態の分布とV関数のドット積を計算して目的関数を求める

def compute_J_c_U(U,Q_pi,c):

    return np.max([compute_J_c_P(P,Q_pi,c) for P in U])  # 各状態遷移確率行列に対して目的関数を計算し、最大値を返す

# print(compute_J_c_P(U[0],Q[0],C[0]))  # 最初の不確実性集合とコスト関数で目的関数を計算
print(compute_J_c_U(U,Q[0],C[0]))  # 不確実性集合とコスト関数で目的関数の最大値を計算

def compute_argmax_n_J_c_P(U, Q, C, B):
    """
    J_c_Uの最大値を持つnを求めます．
    
    Parameters:
    J_c_U : np.ndarray
        目的関数の集合
    b : np.ndarray
        制約条件

    Returns:
    int
        最大値を持つnのインデックス
    """



    temp = []
    for n in range(N):
        temp.append(compute_J_c_P - B[n])




6.05134285379478


In [ ]:
import random
import numpy as np

S = 3
A = 2
N = 4  
U_num = 3

gamma = 0.99

# pi (S*A の1次元ベクトル)
pi = np.zeros((S, A)) 
pi_temp_table = np.random.rand(S, A)
pi_temp_table = pi_temp_table / np.sum(pi_temp_table, axis=1, keepdims=True)
pi = pi_temp_table.flatten() # グローバル変数 pi (1次元 S*A ベクトル)

mu = np.random.rand(S)
mu = mu / np.sum(mu) # グローバル変数 mu

H = int(np.round(1 / (1 - gamma)))

def make_U():
    U_list = []
    for _ in range(U_num):
        P_kernel = np.random.rand(S, A, S)
        P_kernel = P_kernel / np.sum(P_kernel, axis=2, keepdims=True)
        U_list.append(P_kernel)
    return U_list

U = make_U() # グローバル変数 U

def make_C_and_Q_pi_b():
    C_list = []
    Q_list = [] # 元のQリスト (Q_piのリスト)
    B_list = []
    for _ in range(N): # N個のコスト関数、Q行列、閾値を作成
        c_sa = np.random.rand(S, A)
        C_list.append(c_sa)
        Q_pi_initial = np.zeros((S, A)) # 各コスト関数に対応するQの初期値
        Q_list.append(Q_pi_initial)
        b_val = random.randint(0, H)
        B_list.append(b_val)
    return C_list, Q_list, B_list

C, Q_initial_list, B = make_C_and_Q_pi_b() # グローバル変数 C, Q_initial_list, B
                                       # 元の Q は Q_initial_list に改名


def compute_dot_pi_Q(s_idx, current_Q_sa,pi): # Q_pi を current_Q_sa に変更
    """ポリシーとQ関数のドット積 V(s) = sum_a pi(a|s)Q(s,a)"""

    pi_s_actions = pi[s_idx * A : (s_idx + 1) * A]
    return np.dot(pi_s_actions, current_Q_sa[s_idx, :])

def compute_sum_dot_pi_Q(current_Q_sa,pi): # Q_pi を current_Q_sa に変更
    """全てのsについてポリシーとQ関数のドット積を計算した配列"""

    V_s_results = np.zeros(S)
    for s_idx in range(S):
        V_s_results[s_idx] = compute_dot_pi_Q(s_idx, current_Q_sa,pi)
    return V_s_results

def compute_sum_P_dot_dot_pi_Q(s_idx, a_idx, P_kernel_sas, current_Q_sa,pi): # Q_pi を current_Q_sa に変更
    """sum_{s'} P(s'|s,a) V(s') を計算"""
    V_s_prime_current = compute_sum_dot_pi_Q(current_Q_sa,pi) # 現在のQからVを計算
    return np.dot(P_kernel_sas[s_idx, a_idx, :], V_s_prime_current)

def compute_Q_pi_c_P(s_idx, a_idx, P_kernel_sas, current_Q_sa, cost_sa_current,pi): # Q_pi を current_Q_sa, c を cost_sa_current に変更
    """Q(s,a) の1ステップ更新値を計算"""
    expected_future_val = compute_sum_P_dot_dot_pi_Q(s_idx, a_idx, P_kernel_sas, current_Q_sa,pi)
    return cost_sa_current[s_idx, a_idx] + gamma * expected_future_val


# --- Q関数の反復計算を行うように改変された compute_Q_pi_c_P_matrix ---
# 元の変数名: P, Q_pi, c
# P -> P_kernel_sas (遷移確率カーネル)
# Q_pi -> Q_sa_initial_guess (Q行列の初期値、または無視して内部でゼロ初期化)
# c -> cost_sa_current (コスト関数)
def compute_Q_pi_c_P_matrix( # この関数名自体は元のまま
    P_kernel_sas,         # 元の引数 P
    Q_sa_initial_guess,   # 元の引数 Q_pi (初期値として使われる)
    cost_sa_current,      # 元の引数 c
    pi,  # ポリシー
    # グローバル S, A, gamma, pi を参照
    max_iterations=1000,
    tolerance=0.001
    ):
    """
    Q関数の行列を反復計算します。
    グローバル変数 pi, S, A, gamma を参照します。
    引数 Q_sa_initial_guess はQの初期値として使用されます (通常はゼロ行列)。
    """
    # Q_k_sa は反復計算用のQ行列。初期値として引数の Q_sa_initial_guess をコピーして使う。
    # もし常にゼロから始めたい場合は Q_k_sa = np.zeros((S, A)) とする。
    # 元のコードではQ_initial_listの要素(ゼロ行列)が渡されることを想定しているため、コピーが適切。
    Q_k_sa = np.copy(Q_sa_initial_guess)

    for iteration in range(max_iterations):
        Q_k_plus_1_sa = np.zeros((S, A)) # 次のステップのQ値を格納する行列
        for s_idx in range(S):
            for a_idx in range(A):
                # compute_Q_pi_c_P は現在の Q_k_sa を使って Q_{k+1}(s,a) の値を計算
                Q_k_plus_1_sa[s_idx, a_idx] = compute_Q_pi_c_P( # 元の compute_Q_pi_c_P を呼び出し
                    s_idx, a_idx, P_kernel_sas, Q_k_sa, cost_sa_current,pi
                )
        
        # 収束判定
        if np.max(np.abs(Q_k_plus_1_sa - Q_k_sa)) < tolerance:
            print(f"Q converged at iteration {iteration + 1} for P: {P_kernel_sas[0,0,0]:.2f} c: {cost_sa_current[0,0]:.2f}")
            return Q_k_plus_1_sa # 収束したQ行列を返す
        Q_k_sa = Q_k_plus_1_sa # Q値を更新
        
    print(f"Warning: Q-function did not converge in {max_iterations} iterations for P: {P_kernel_sas[0,0,0]:.2f} c: {cost_sa_current[0,0]:.2f}")
    
    return Q_k_sa # 収束しなかった場合は最後のQ行列を返す

def compute_J_c_P(P_kernel_sas, Q_sa_initial_guess, cost_sa_current,pi):
    """
    目的関数 J_c_P を計算します。
    
    Parameters:
    P_kernel_sas : np.ndarray
        状態遷移確率行列
    Q_sa_initial_guess : np.ndarray
        Q行列の初期値
    cost_sa_current : np.ndarray
        コスト関数

    Returns:
    float
        目的関数の値
    """
    Q_sa = compute_Q_pi_c_P_matrix(P_kernel_sas, Q_sa_initial_guess, cost_sa_current,pi)  # Q行列を計算
    V_s = np.zeros(S)  # V関数の初期化
    pi_matrix = pi.reshape(S, A)  # ポリシーをS行A列の形に変形
    for s_idx in range(S):
        V_s[s_idx] = np.dot(pi_matrix[s_idx, :], Q_sa[s_idx, :])  # 各状態のV関数を計算
    return np.dot(mu, V_s)  # 初期状態の分布とV関数のドット積を計算して目的関数を求める



def compute_Q_pi_for_all_U(C, Q_initial_list,U):
    """
    全ての不確実性集合に対してQ関数を計算します。
    
    Parameters:
    C : list of np.ndarray
        コスト関数のリスト
    Q_initial_list : list of np.ndarray
        Q行列の初期値のリスト

    Returns:
    list of np.ndarray
        各不確実性集合に対するQ行列のリスト
    """
    Q_cn_U_results = []
    Q_results_respect_U = []  # Q関数の結果を格納するリスト
    for j in range(N):
        for i in range(U_num):
            Q_result = compute_Q_pi_c_P_matrix(U[i], Q_initial_list[i], C[j])
            Q_results_respect_U.append(Q_result)
        Q_results = np.array(Q_results_respect_U)  # リストをNumPy配列に変換
        Q_results = np.max(Q_results, axis=0)  # 各状態の最大Q値を計算
        Q_cn_U_results.append(Q_results)  # 結果をリストに追加
    return Q_cn_U_results  # 各不確実性集合に対するQ行列のリストを返す

def compute_JcnU_b_and_its_max(O_cn_U_results, B):
    """
    ロバスト目的関数を計算し、最大値を返します。
    
    Parameters:
    O_cn_U_results : list of np.ndarray
        各不確実性集合に対するQ行列のリスト
    B : list of float
        制約条件のリスト

    Returns:
    tuple
        ロバスト目的関数の値とその最大値
    """





# print(compute_Q_pi_for_all_U(C, Q_initial_list, U))  # 全ての不確実性集合に対してQ関数を計算

'''
ここまでできたことは，Qの行列を計算するところです．
次にすることは，目的関数の計算です．
その次にはロバスト目的関数を計算するコードを書きましょう．入力はCとUです．
その次は，C全部を使って，ロバスト目的関数-bnを計算しましょう．そこで一番大きな値をnとしてとりだします．
ロバスト目的関数の勾配を計算するコードを書きましょう．
最小化問題を解くコードを書きましょう．
内側ループの完成です．
'''





Q converged at iteration 597 for P: 0.45 c: 0.29
39.51510039685908


'\nここまでできたことは，Qの行列を計算するところです．\n次にすることは，目的関数の計算です．\nその次にはロバスト目的関数を計算するコードを書きましょう．入力はCとUです．\nその次は，C全部を使って，ロバスト目的関数-bnを計算しましょう．そこで一番大きな値をnとしてとりだします．\nロバスト目的関数の勾配を計算するコードを書きましょう．\n最小化問題を解くコードを書きましょう．\n内側ループの完成です．\n'

### 得られた知見
内側ルーチンでのベルマン方程式の計算は，収束するまで反復的に計算をしている．
* 価値反復法は1反復で1ステップの更新だけ
* 方策反復法は1反復で収束するまで反復更新をする